In [1]:
import sqlite3
import pandas as pd


In [4]:
%load_ext sql


In [7]:
%sql sqlite:///badge_attendance.db

In [8]:
%%sql
SELECT COUNT(*) FROM src_badge_events;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
1000000


In [9]:
%%sql
CREATE TABLE IF NOT EXISTS fact_attendance_monthly (
    fact_id INTEGER PRIMARY KEY AUTOINCREMENT,
    badge_id TEXT,
    person_type TEXT,
    year INTEGER,
    month INTEGER,
    total_hours REAL,
    session_count INTEGER
);


 * sqlite:///badge_attendance.db
Done.


[]

In [10]:
%%sql
DELETE FROM fact_attendance_monthly;


 * sqlite:///badge_attendance.db
0 rows affected.


[]

In [11]:
%%sql
WITH ordered_events AS (
    SELECT
        badge_id,
        person_type,
        event_time,
        event_type,
        LEAD(event_time) OVER (
            PARTITION BY badge_id
            ORDER BY event_time
        ) AS next_time
    FROM src_badge_events
),
sessions AS (
    SELECT
        badge_id,
        person_type,
        event_time AS in_time,
        next_time AS out_time
    FROM ordered_events
    WHERE event_type = 'IN'
)
SELECT
    badge_id,
    person_type,
    in_time,
    out_time,
    (julianday(out_time) - julianday(in_time)) * 24 AS session_hours
FROM sessions
WHERE out_time IS NOT NULL
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,person_type,in_time,out_time,session_hours
BADGE000000,student,2024-09-01T13:28:00,2024-09-01T15:10:00,1.699999999254942
BADGE000000,student,2024-09-01T15:10:00,2024-09-01T16:13:00,1.0499999932944775
BADGE000000,student,2024-09-02T11:27:00,2024-09-02T14:21:00,2.9000000059604645
BADGE000000,student,2024-09-02T14:21:00,2024-09-02T15:01:00,0.6666666604578495
BADGE000000,student,2024-09-02T16:50:00,2024-09-02T17:50:00,0.9999999962747097
BADGE000000,student,2024-09-03T10:52:00,2024-09-03T13:31:00,2.649999998509884
BADGE000000,student,2024-09-04T11:21:00,2024-09-04T12:07:00,0.7666666656732559
BADGE000000,student,2024-09-05T16:20:00,2024-09-05T16:39:00,0.3166666701436043
BADGE000000,student,2024-09-06T15:02:00,2024-09-06T16:29:00,1.4500000029802322
BADGE000000,student,2024-09-07T12:12:00,2024-09-07T15:19:00,3.1166666708886623


In [12]:
%%sql
INSERT INTO fact_attendance_monthly (
    badge_id,
    person_type,
    year,
    month,
    total_hours,
    session_count
)
WITH ordered_events AS (
    SELECT
        badge_id,
        person_type,
        event_time,
        event_type,
        LEAD(event_time) OVER (
            PARTITION BY badge_id
            ORDER BY event_time
        ) AS next_time
    FROM src_badge_events
),
sessions AS (
    SELECT
        badge_id,
        person_type,
        event_time AS in_time,
        next_time AS out_time
    FROM ordered_events
    WHERE event_type = 'IN'
),
session_durations AS (
    SELECT
        badge_id,
        person_type,
        CAST(strftime('%Y', in_time) AS INTEGER) AS year,
        CAST(strftime('%m', in_time) AS INTEGER) AS month,
        (julianday(out_time) - julianday(in_time)) * 24 AS session_hours
    FROM sessions
    WHERE out_time IS NOT NULL
)
SELECT
    badge_id,
    person_type,
    year,
    month,
    ROUND(SUM(session_hours), 2) AS total_hours,
    COUNT(*) AS session_count
FROM session_durations
GROUP BY badge_id, person_type, year, month;


 * sqlite:///badge_attendance.db
17199 rows affected.


[]

In [14]:
%%sql
SELECT * FROM fact_attendance_monthly LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


fact_id,badge_id,person_type,year,month,total_hours,session_count
1,BADGE000000,student,2024,9,66.15,31
2,BADGE000000,student,2024,10,60.58,28
3,BADGE000000,student,2024,11,53.37,30
4,BADGE000000,student,2024,12,50.75,26
5,BADGE000000,student,2025,1,83.35,37
6,BADGE000000,student,2025,2,36.8,23
7,BADGE000000,student,2025,3,44.7,25
8,BADGE000000,student,2025,4,25.12,11
9,BADGE000001,student,2024,9,54.55,24
10,BADGE000001,student,2024,10,67.75,29


In [15]:
%%sql
SELECT badge_id, total_hours
FROM fact_attendance_monthly
WHERE person_type = 'student'
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,total_hours
BADGE000959,100.67
BADGE000958,100.35
BADGE001218,98.33
BADGE001057,98.15
BADGE000302,97.93
BADGE000004,97.0
BADGE000435,96.98
BADGE001047,95.78
BADGE001109,95.68
BADGE000283,94.77


In [19]:
%%sql
SELECT
    s.name,
    f.badge_id,
    f.total_hours
FROM fact_attendance_monthly f
JOIN dim_students s
    ON f.badge_id = s.badge_id
WHERE f.person_type = 'student' 
  AND f.year = 2025
ORDER BY f.total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,badge_id,total_hours
Barbara Kim,BADGE000959,100.67
Andrew Gaines,BADGE001057,98.15
Angela Brewer,BADGE001109,95.68
Jeff Williams,BADGE000644,94.73
Tonya Green,BADGE001874,94.38
Blake Hale,BADGE001986,94.0
William Pham,BADGE000756,93.72
Shannon Bowen,BADGE000867,93.47
Michael Santos,BADGE001177,93.38
Peter Ramsey,BADGE001877,93.38


In [20]:
%%sql
SELECT
    t.name,
    f.badge_id,
    f.total_hours
FROM fact_attendance_monthly f
JOIN dim_teachers t
    ON f.badge_id = t.badge_id
WHERE f.person_type = 'teacher'
  AND f.year=2025
ORDER BY f.total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,badge_id,total_hours
Terry Mendez,TEACHER00117,139.82
William Booth,TEACHER00055,131.6
Mark Andrews,TEACHER00076,126.07
Leslie Holmes,TEACHER00048,124.73
Daniel Bender,TEACHER00128,122.65
Diana Lopez DDS,TEACHER00072,121.62
Mrs. Jennifer Guzman,TEACHER00023,120.9
Charles Perry,TEACHER00080,120.52
Amy Caldwell,TEACHER00052,120.07
Jeffrey Perry,TEACHER00106,119.42


In [22]:
%%sql
SELECT
    t.name,
    t.department AS subject,
    f.total_hours
FROM fact_attendance_monthly f
JOIN dim_teachers t
    ON f.badge_id = t.badge_id
WHERE f.person_type = 'teacher'
AND f.year= 2025
ORDER BY f.total_hours ASC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,subject,total_hours
Christina Cook,Business,9.92
Brenda Frey,Engineering,10.33
Nathan Perkins,Arts,11.57
Lisa Miller,Engineering,12.03
David Davis,Law,14.4
Maureen Wright,Medicine,14.6
Theodore Browning,Science,14.9
Sarah Garcia,Medicine,15.15
Thomas Campbell,Arts,15.3
Jonathan Dominguez,Medicine,16.37
